In [1]:
import os
import json
import gzip
from pyBioInfo.IO.File import BedFile, GtfFile, GtfGeneBuilder, GtfTranscriptBuilder
from pyBioInfo.Utils import ShiftLoader
import numpy as np
import pandas as pd
import pysam
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import rcParams
samples = ["Ad_Fe", "Ad_Ma", "Ju_Mi"]

# Subreads

In [39]:
# Ad_Fe
counter = defaultdict(int)
with open("data/rawdata/F20FTSCCKF1558_CIYgikE/GTS-E-14-1A/GTS-E-14-1A_subreads_dis.xls") as f:
    for line in f:
        counter[line.split("/")[1]] += 1
vs = list(counter.values())
print(np.mean(vs), np.std(vs), np.median(vs), sep="\t")

15.344987979567348	28.133017426153327	9.0


In [40]:
# Ad_Ma
counter = defaultdict(int)
with open("data/rawdata/F20FTSCCKF0616_SERmjgE_SERsukE/WHPBCDNAPEP00000149/WHPBCDNAPEP00000149_subreads_dis.xls") as f:
    for line in f:
        counter[line.split("/")[1]] += 1
vs = list(counter.values())
print(np.mean(vs), np.std(vs), np.median(vs), sep="\t")

15.752177268978274	25.761194842360247	9.0


In [41]:
# Ju_Mi
counter = defaultdict(int)
with open("data/rawdata/F20FTSCCKF0616_SERdweE_SERvxlE/WHPBCDNAPEP00000150/WHPBCDNAPEP00000150_subreads_dis.xls") as f:
    for line in f:
        counter[line.split("/")[1]] += 1
vs = list(counter.values())
print(np.mean(vs), np.std(vs), np.median(vs), sep="\t")

14.697232628597908	24.101903104782103	7.0


# CCS lengths

In [20]:
for name in ["Ju_Mi", "Ad_Fe", "Ad_Ma"]:
    path = "results/isoseq/ccs/%s.bam" % name
    lengths = []
    with pysam.AlignmentFile(path, check_sq=False) as f:
        for segment in f:
            lengths.append(segment.query_length)
    vs = list(sorted(lengths, reverse=True))
    total = sum(vs)
    half = total / 2
    tmp = 0
    n50 = 0
    for v in vs:
        tmp += v
        if tmp >= half:
            n50 = v
            break
            
    print(name, np.mean(vs), np.std(vs), np.median(vs), n50, vs[-1], vs[0], sep="\t")

Ju_Mi	4829.946254855643	1739.370156067987	5358.0	5541	200	25638
Ad_Fe	5700.600120896488	1409.1814635401736	5887.0	6031	200	27347
Ad_Ma	5115.969669346235	1380.4876747471856	5305.0	5369	200	28779


# High-quality transcripts

In [28]:
from Bio import SeqIO

for name in ["Ju_Mi", "Ad_Fe", "Ad_Ma"]:
    path = "results/isoseq/polished/%s.hq.fasta.gz" % name
    lengths = []
    with gzip.open(path, "rt") as f:
        for record in SeqIO.parse(f, "fasta"):
            lengths.append(len(record))
    vs = list(sorted(lengths, reverse=True))
    total = sum(vs)
    half = total / 2
    tmp = 0
    n50 = 0
    for v in vs:
        tmp += v
        if tmp >= half:
            n50 = v
            break
    print(name, len(lengths), np.mean(vs), np.std(vs), np.median(vs), n50, vs[-1], vs[0], sep="\t")

Ju_Mi	163218	4896.718002916345	1550.7981043173784	5310.0	5441	59	10176
Ad_Fe	189169	5670.282181541373	1396.4887086735723	5798.0	5939	52	12381
Ad_Ma	149716	4903.359680996019	1175.5636824919668	5210.0	5255	59	11121


In [40]:
array = []
for sample in samples:
    s = pd.Series(dtype=np.object)
    s.name = sample
    with open("results/isoseq/ccs/%s.ccs_report.txt" % sample) as f:
        for line in f:
            line = line.strip("\n")
            if line == "" or ":" not in line:
                continue
            row = line.split(":")
            if len(row) != 2:
                continue
            k = row[0].strip()
            v = row[1].strip()
            if v == "":
                continue
            s[k] = v
            # print(k, v, sep="\t")
    with open("results/isoseq/demux/%s.lima.summary" % sample) as f:
        for line in f:
            line = line.strip("\n")
            if ":" not in line:
                continue
            row = line.split(":")
            if len(row) != 2:
                continue
            k, v = row[0].strip(), row[1].strip()
            if v == "":
                continue
            s[k] = v
    with open("results/isoseq/flnc/%s.filter_summary.json" % sample) as f:
        d = json.loads(f.read())
        for k, v in d.items():
            s[k] = v
    array.append(s)
    # break

In [41]:
dat = pd.DataFrame(array)
dat

,ZMWs input,ZMWs pass filters,ZMWs fail filters,ZMWs shortcut filters,ZMWs with tandem repeats,Below SNR threshold,Median length filter,Lacking full passes,Heteroduplex insertions,Coverage drops,...,Undesired no hit,With different pair,Coefficient of correlation,Allow diff pair,Allow same pair,Above length,Below length,num_reads_fl,num_reads_flnc,num_reads_flnc_polya
Ad_Fe,5760192,3364862 (58.42%),2395330 (41.58%),0 (0.00%),4351 (0.08%),107556 (4.49%),0 (0.00%),2233973 (93.26%),7107 (0.30%),1140 (0.05%),...,26 (0%),2424815 (100%),0%,3364836 (100%),3364836 (100%),2424815 (100%),0 (0%),2424815,2388600,2381123
Ad_Ma,5215938,2967394 (56.89%),2248544 (43.11%),0 (0.00%),5996 (0.11%),184132 (8.19%),0 (0.00%),2020953 (89.88%),6726 (0.30%),963 (0.04%),...,13 (0%),2073557 (100%),0%,2967381 (100%),2967381 (100%),2073557 (100%),0 (0%),2073557,1895658,1887924
Ju_Mi,6433289,3452684 (53.67%),2980605 (46.33%),0 (0.00%),4267 (0.07%),191504 (6.43%),0 (0.00%),2712150 (90.99%),10232 (0.34%),1524 (0.05%),...,29 (0%),2464758 (100%),0%,3452655 (100%),3452655 (100%),2464758 (100%),0 (0%),2464758,2348576,2339901


# 2021-03-24 Summary

In [358]:
def get_raw_read(sample):
    path = "results/isoseq/ccs/%s.ccs_report.txt" % sample
    with open(path) as f:
        for line in f:
            if "ZMWs input" in line:
                return int(line.strip("\n").split()[-1])
            
def get_ccs_read(sample):
    path = "results/isoseq/ccs/%s.ccs_report.txt" % sample
    with open(path) as f:
        for line in f:
            if "ZMWs pass filters" in line:
                return int(line.strip("\n").split()[-2])

def get_lima_read(sample):
    path = "results/isoseq/demux/%s.lima.summary" % sample
    with open(path) as f:
        for line in f:
            if "ZMWs above all thresholds" in line:
                return int(line.strip("\n").split()[-2])
    
def get_flnc_read(sample):
    path = "results/isoseq/flnc/%s.filter_summary.json" % sample
    with open(path) as f:
        for line in f:
            js = json.loads(line)
            return js["num_reads_flnc"]
        
def get_flnc_polya_read(sample):
    path = "results/isoseq/flnc/%s.filter_summary.json" % sample
    with open(path) as f:
        for line in f:
            js = json.loads(line)
            return js["num_reads_flnc_polya"]
        
def get_cluster(sample):
    path = "results/isoseq/clustered/%s.cluster_report.csv" % sample
    # return 0
    array = []
    with open(path) as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            array.append(line.strip("\n").split(",")[0])
    return len(set(array))

def get_aligned(sample):
    path = "results/isoseq/aligned/%s.stats" % sample
    with open(path) as f:
        for line in f:
            if "Total reads:" in line:
                return int(line.strip("\n").split()[-1])
    
dat = pd.DataFrame(index=pd.Index(samples, name="Sample"))
dat["RawRead"] = list(map(get_raw_read, samples))
dat["CCS"] = list(map(get_ccs_read, samples))
dat["Lima"] = list(map(get_lima_read, samples))
dat["FLNC"] = list(map(get_flnc_read, samples))
dat["FLNC_polyA"] = list(map(get_flnc_polya_read, samples))
dat["Cluster"] = list(map(get_cluster, samples))
dat["Aligned"] = list(map(get_aligned, samples))
dat

,RawRead,CCS,Lima,FLNC,FLNC_polyA,Cluster,Aligned
Sample,,,,,,,
Adult_Female,5759266,3971860,2767549,2725947,2706469,213849,219085
Adult_Male,5215426,3509670,2364704,2160279,2142476,170298,173656
Juvenile,6432352,4133860,2821785,2691254,2666770,188165,190721


In [359]:
dat.to_csv("summary.tsv", sep="\t")

# 2021-03-24 Cluster profile

In [354]:
for sample in samples:
    counter = defaultdict(int)
    with open("results/isoseq/clustered/%s.cluster_report.csv" % sample) as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            name = line.strip("\n").split(",")[0]
            counter[name] += 1
    counter = Counter(counter.values())
    
    xs = np.arange(0, 20)
    ys = [counter[x] for x in xs]
    ys = np.array(ys) / 1000
    
    plt.figure(figsize=(3, 3))
    plt.bar(xs, ys, width=0.8, color="dodgerblue")
    plt.xlim(0, 20)
    plt.xlabel("FLNC per cluster")
    plt.ylim(0, 120)
    plt.ylabel("Number (x10$^3$)")
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["right"].set_visible(False)
    plt.tight_layout()
    # plt.show()
    plt.savefig("figures/cluster.profile.%s.png" % sample, dpi=300)
    plt.close()

# 2021-03-24 Plot length distribution

In [4]:
def run(path1, path2, prefix):
    polymerase_lengths = dict()
    with open(path1) as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            name, length = line.strip("\n").split("\t")
            polymerase_lengths[name] = int(length)

    rows = []
    with open(path2) as f:
        for line in f:
            v1, v2, v3 = line.strip("\n").split("\t")
            v2 = int(v2)
            rows.append([v1, v2])

    container = defaultdict(list)
    for v1, v2 in rows:
        name = v1.split("/")[1]
        container[name].append(v2)

    matrix = []
    for zmw, values in container.items():
        pl = polymerase_lengths[zmw]
        sl = np.median(values)
        n = len(values)
        matrix.append([pl, sl, n])

    temp = matrix
    vs1 = np.log10([vs[0] for vs in temp]) # polymerase
    vs2 = np.log10([vs[1] for vs in temp]) # subreads
    vs3 = np.log2([vs[2]for vs in temp])   # subread count
    
    m = np.zeros((600, 600))
    for x, y in zip(vs1, vs2):
        i = int(x * 100)
        j = int(y * 100)
        if i < 600 and j < 600:
            m[i][j] += 1
    plt.figure(figsize=(3, 3))
    plt.hist2d(vs1, vs2, bins=(600, 600), cmap="Reds", norm=colors.LogNorm(1, m.max()))
    plt.xlim(0, 6)
    plt.xlabel("Polymerase length")
    xs = [0, 1, 2, 3, 4, 5, 6]
    plt.xticks(xs, ["10$^{%d}$" % x for x in xs])
    plt.ylim(0, 6)
    ys = [0, 1, 2, 3, 4, 5, 6]
    plt.yticks(ys, ["10$^{%d}$" % y for y in ys])
    plt.ylabel("Subread length")
    plt.tight_layout()
    # plt.show()
    plt.savefig(prefix + ".1.png", dpi=300)
    plt.close()
    
    m = np.zeros((600, 1000))
    for x, y in zip(vs1, vs3):
        i = int(x * 100)
        j = int(y * 100)
        if i < 600 and j < 1000:
            m[i][j] += 1
    plt.figure(figsize=(3, 3))
    plt.hist2d(vs1, vs3, bins=(600, 1000), cmap="Reds", norm=colors.LogNorm(1, m.max()))
    plt.xlim(0, 6)
    plt.xlabel("Polymerase length")
    xs = [0, 1, 2, 3, 4, 5, 6]
    plt.xticks(xs, ["10$^{%d}$" % x for x in xs])
    plt.ylim(-1, 10)
    ys = np.arange(0, 11, 2)
    plt.yticks(ys, [2 ** y for y in ys])
    plt.ylabel("Subread count")
    # plt.yticks([0, 2, 4, 6])
    plt.tight_layout()
    # plt.show()
    plt.savefig(prefix + ".2.png", dpi=300)
    plt.close()
    
    m = np.zeros((600, 1000))
    for x, y in zip(vs2, vs3):
        i = int(x * 100)
        j = int(y * 100)
        if i < 600 and j < 1000:
            m[i][j] += 1
    plt.figure(figsize=(3, 3))
    plt.hist2d(vs2, vs3, bins=(600, 1000), cmap="Reds", norm=colors.LogNorm(1, m.max()))
    plt.xlim(0, 6)
    plt.xlabel("Subread length")
    xs = [0, 1, 2, 3, 4, 5, 6]
    plt.xticks(xs, ["10$^{%d}$" % x for x in xs])
    plt.ylim(-1, 10)
    ys = np.arange(0, 11, 2)
    plt.yticks(ys, [2 ** y for y in ys])
    plt.ylabel("Subread count (median)")
    # plt.yticks([0, 2, 4, 6])
    plt.tight_layout()
    # plt.show()
    plt.savefig(prefix + ".3.png", dpi=300)
    plt.close()

    xs = np.arange(100)
    ys = np.zeros(100)
    for row in matrix:
        i = int(row[0] / 1000)
        if i < 100:
            ys[i] += 1
    m1 = np.mean([vs[0] for vs in matrix])
    m2 = np.median([vs[0] for vs in matrix])
    ys = ys / 1000
    plt.figure(figsize=(3, 3))
    plt.bar(xs + 0.5, ys, width=1, color="dodgerblue")
    plt.text(20, 180, "width = 1000bp")
    plt.text(20, 160, "mean = %.2fkb" % (m1 / 1000))
    plt.text(20, 140, "median = %.2fkb" % (m2 / 1000))
    plt.xlim(0, 100)
    vs = [0, 20, 40, 60, 80, 100]
    plt.xticks(vs)
    plt.xlabel("Polymerase length (kb)")
    plt.ylim(0, 200)
    plt.yticks([0, 50, 100, 150, 200])
    plt.ylabel("Number (x10$^3$)")
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["right"].set_visible(False)
    plt.tight_layout()
    # plt.show()
    plt.savefig(prefix + ".4.png", dpi=300)
    plt.close()

    xs = np.arange(100)
    ys = np.zeros(100)
    for row in matrix:
        i = int(row[1] / 100)
        if i < 100:
            ys[i] += 1
    ys = ys / 1000
    m1 = np.mean([vs[1] for vs in matrix])
    m2 = np.median([vs[1] for vs in matrix])
    plt.figure(figsize=(3, 3))
    plt.bar(xs + 0.5, ys, width=1, color="dodgerblue")
    plt.text(10, 380, "width = 100bp")
    plt.text(10, 345, "mean = %.2fkb" % (m1 / 1000))
    plt.text(10, 310, "median = %.2fkb" % (m2 / 1000))
    plt.xlim(0, 100)
    vs = [0, 20, 40, 60, 80, 100]
    plt.xticks(vs, [int(v / 10) for v in vs])
    plt.xlabel("Subread length (kb)")
    plt.ylim(0, 400)
    plt.yticks([0, 100, 200, 300, 400])
    plt.ylabel("Number (x10$^3$)")
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["right"].set_visible(False)
    plt.tight_layout()
    # plt.show()
    plt.savefig(prefix + ".5.png", dpi=300)
    plt.close()

    xs = np.arange(20)
    ys = np.zeros(20)
    for row in matrix:
        i = row[2]
        if i < 20:
            ys[i] += 1
    ys = ys / 1000000
    plt.figure(figsize=(3, 3))
    plt.bar(xs, ys, width=0.8, color="dodgerblue")
    # plt.text(10, 380, "width = 100bp")
    # plt.text(10, 345, "mean = %.2fkb" % (m1 / 1000))
    # plt.text(10, 310, "median = %.2fkb" % (m2 / 1000))
    plt.xlim(0, 20)
    # vs = [0, 20, 40, 60, 80, 100]
    # plt.xticks(vs, [int(v / 10) for v in vs])
    plt.xlabel("Subread number")
    plt.ylim(0, 1.6)
    plt.yticks([0, 0.4, 0.8, 1.2, 1.6])
    plt.ylabel("Number (x10$^6$)")
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["right"].set_visible(False)
    plt.tight_layout()
    # plt.show()
    plt.savefig(prefix + ".6.png", dpi=300)
    plt.close()

In [5]:
path1 = "data/datasets/F20FTSCCKF1558_CIYgikE/GTS-E-14-1A/GTS-E-14-1A.polymerreads_dis.xls"
path2 = "data/datasets/F20FTSCCKF1558_CIYgikE/GTS-E-14-1A/GTS-E-14-1A_subreads_dis.xls"
prefix = "figures/length.dis.adult.female"
run(path1, path2, prefix)

In [7]:
path1 = "data/datasets/F20FTSCCKF0616_SERmjgE_SERsukE/WHPBCDNAPEP00000149/WHPBCDNAPEP00000149.polymerreads_dis.xls"
path2 = "data/datasets/F20FTSCCKF0616_SERmjgE_SERsukE/WHPBCDNAPEP00000149/WHPBCDNAPEP00000149_subreads_dis.xls"
prefix = "figures/length.dis.adult.male"
run(path1, path2, prefix)

In [8]:
path1 = "data/datasets/F20FTSCCKF0616_SERdweE_SERvxlE/WHPBCDNAPEP00000150/WHPBCDNAPEP00000150.polymerreads_dis.xls"
path2 = "data/datasets/F20FTSCCKF0616_SERdweE_SERvxlE/WHPBCDNAPEP00000150/WHPBCDNAPEP00000150_subreads_dis.xls"
prefix = "figures/length.dis.juvenile"
run(path1, path2, prefix)

In [11]:
# 2021-03-24 

In [16]:
from pyBioInfo.IO.File import GtfFile, GtfTranscriptBuilder, GtfGeneBuilder

In [36]:
path = "data/genome/annotation.gtf"
with GtfFile(path) as f:
    records = [x for x in f] 
genes = list(GtfGeneBuilder(records))
counter = Counter([len(gene.transcripts) for gene in genes])
# for i in range(0, 50):
#     print(i, counter[i])
print(np.mean([len(gene.transcripts) for gene in genes]))

1.458123399686132


In [34]:
path = "results/cupcake/chain/all_samples.chained.gtf.gz"
with GtfFile(path) as f:
    records = [x for x in f] 
genes = list(GtfGeneBuilder(records))
counter = Counter([len(gene.transcripts) for gene in genes])
for i in range(0, 50):
    print(i, counter[i])

0 0
1 12789
2 5582
3 3440
4 2372
5 1747
6 1427
7 1158
8 953
9 762
10 652
11 557
12 540
13 429
14 371
15 319
16 297
17 264
18 210
19 209
20 187
21 160
22 149
23 145
24 121
25 108
26 82
27 99
28 85
29 79
30 83
31 64
32 59
33 57
34 41
35 54
36 51
37 34
38 37
39 45
40 43
41 29
42 25
43 24
44 32
45 34
46 26
47 14
48 28
49 15


In [35]:
print(np.mean([len(gene.transcripts) for gene in genes]))

5.948429815644147


# 2021-03-24 CCS and lima report

In [315]:
for sample in ["Adult_Female", "Adult_Male", "Juvenile"]:
    path = "results/isoseq/ccs/%s.ccs_report.txt" % sample
    print(path)
    with open(path) as f:
        for line in f:
            if ":" not in line:
                continue
            v1, v2 = line.strip("\n").split(":")
            v1 = v1.strip()
            v2 = v2.strip()
            if v2 == "":
                continue
            print(v2)

results/isoseq/ccs/Adult_Female.ccs_report.txt
5759266
3971860 (68.96%)
1787406 (31.04%)
0 (0.00%)
10137 (0.18%)
107556 (6.02%)
0 (0.00%)
1426910 (79.83%)
16073 (0.90%)
1382 (0.08%)
184282 (10.31%)
1 (0.00%)
51179 (2.86%)
926 (0.05%)
23 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)
results/isoseq/ccs/Adult_Male.ccs_report.txt
5215426
3509670 (67.29%)
1705756 (32.71%)
0 (0.00%)
11358 (0.22%)
184132 (10.79%)
0 (0.00%)
1310618 (76.84%)
14550 (0.85%)
1115 (0.07%)
156561 (9.18%)
0 (0.00%)
38760 (2.27%)
512 (0.03%)
20 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)
results/isoseq/ccs/Juvenile.ccs_report.txt
6432352
4133860 (64.27%)
2298492 (35.73%)
0 (0.00%)
11022 (0.17%)
191504 (8.33%)
0 (0.00%)
1772906 (77.13%)
22851 (0.99%)
1758 (0.08%)
237562 (10.34%)
2 (0.00%)
71812 (3.12%)
937 (0.04%)
97 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)
0 (0.00%)


In [318]:
for sample in ["Adult_Female", "Adult_Male", "Juvenile"]:
    path = "results/isoseq/demux/%s.lima.summary" % sample
    print(path)
    with open(path) as f:
        for line in f:
            if ":" not in line:
                continue
            v1, v2 = line.strip("\n").split(":")
            v1 = v1.strip()
            v2 = v2.strip()
            if v2 == "":
                continue
            print(v2)

results/isoseq/demux/Adult_Female.lima.summary
3971860
2767549 (70%)
1204311 (30%)
68 (0%)
0 (0%)
219872 (18%)
61 (0%)
0 (0%)
259986 (22%)
61 (0%)
382455 (32%)
662511 (55%)
61 (0%)
2767549 (100%)
0%
3971799 (100%)
3971799 (100%)
2767549 (100%)
0 (0%)
results/isoseq/demux/Adult_Male.lima.summary
3509670
2364704 (67%)
1144966 (33%)
155 (0%)
0 (0%)
202707 (18%)
120 (0%)
0 (0%)
226676 (20%)
120 (0%)
395688 (35%)
609201 (53%)
120 (0%)
2364704 (100%)
0%
3509550 (100%)
3509550 (100%)
2364704 (100%)
0 (0%)
results/isoseq/demux/Juvenile.lima.summary
4133860
2821785 (68%)
1312075 (32%)
143 (0%)
0 (0%)
300944 (23%)
128 (0%)
0 (0%)
324391 (25%)
128 (0%)
460864 (35%)
641491 (49%)
128 (0%)
2821785 (100%)
0%
4133732 (100%)
4133732 (100%)
2821785 (100%)
0 (0%)


# 2021-03-25 Cupcake report

In [15]:
data = dict()
for sample in samples:
    data[sample] = dict()
    # raw
    path = "results/cupcake/collapsed/%s/out.collapsed.sorted.gtf.gz" % sample
    with GtfFile(path) as f:
        records = [x for x in f]
    genes = list(GtfGeneBuilder(records))
    data[sample]["raw"] = genes
    # filtered
    path = "results/cupcake/collapsed/%s/out.collapsed.filtered.sorted.gtf.gz" % sample
    with GtfFile(path) as f:
        records = [x for x in f]
    genes = list(GtfGeneBuilder(records))
    data[sample]["filtered"] = genes

In [18]:
rows = []
for sample in samples:
    values = []
    genes = data[sample]["raw"]
    values.append(len(genes))
    values.append(sum([len(g.transcripts) for g in genes]))
    exon_count = 0
    for g in genes:
        for t in g.transcripts:
            exon_count += len(t.blocks)
    values.append(exon_count)
    counts = [len(g.transcripts) for g in genes]
    values.append(np.mean(counts))
    values.append(np.median(counts))
    
    genes = data[sample]["filtered"]
    values.append(len(genes))
    values.append(sum([len(g.transcripts) for g in genes]))
    exon_count = 0
    for g in genes:
        for t in g.transcripts:
            exon_count += len(t.blocks)
    values.append(exon_count)
    counts = [len(g.transcripts) for g in genes]
    values.append(np.mean(counts))
    values.append(np.median(counts))
 
    rows.append(values)

In [22]:
columns = [
    "Gene_Raw", "Transcript_Raw", "Exon_Raw", "TranNum_Mean_Raw", "TranNum_Median_Raw",
    "Gene_Filtered", "Transcript_Filtered", "Exon_Filtered", "TranNum_Mean_Filtered", "TranNum_Median_Filtered"
]
dat = pd.DataFrame(rows, index=pd.Index(samples, name="Sample"), columns=columns)
dat.to_csv("cupcake.report.tsv", sep="\t")

In [26]:
dat["Transcript_Filtered"] / dat["Transcript_Raw"] * 100

Sample
Adult_Female    74.833222
Adult_Male      75.626835
Juvenile        69.308592
dtype: float64

In [47]:
plt.figure(figsize=(6, 3))
for i, sample in enumerate(samples):
    counts = [len(g.transcripts) for g in data[sample]["filtered"]]
    counter = Counter(counts)
    xs = np.arange(10)
    ys = [counter[x] for x in xs]
    ys.append(len(counts) - sum(ys))
    xs = np.arange(len(xs) + 1)
    ys = np.array(ys) / len(counts) * 100
    plt.bar(xs + (i - 1) * 0.2, ys, width=0.2, label=sample + " (%.2f)" % np.mean(counts))
plt.xlim(0.5, 10.5)
plt.xticks(np.arange(1, 11))
plt.xlabel("transcripts per gene")
plt.ylim(0, 50)
plt.ylabel("Percentage (%)")
plt.legend()
plt.tight_layout()
if False:
    plt.show()
else:
    plt.savefig("figures/cupcake.summary.png", dpi=300)
plt.close()

In [ ]:
# 

In [10]:
names = ["cupcake", "tama"]
data = dict()
for name in names:
    counts = []
    path = "results/collect/%s.isoform.txt" % name
    with open(path) as f:
        for line in f:
            gid, count = line.strip("\n").split("\t")
            count = int(count)
            counts.append(count)
    data[name] = counts
    

In [44]:
s1 = np.sum(data["cupcake"])
s2 = np.sum(data["tama"])
mean1 = np.mean(data["cupcake"])
median1 = np.median(data["cupcake"])
mean2 = np.mean(data["tama"])
median2 = np.median(data["tama"])
s1, s2, mean1, median1, mean2, median2

(216505, 214112, 12.83372851215175, 8.0, 9.908464065898468, 3.0)

In [49]:
plt.figure(figsize=(7, 4))
for i, name in enumerate(names):
    counts = data[name]
    counter = Counter(counts)
    xs = np.arange(1, 31)
    xticks = [str(x) for x in xs]
    xticks[-1] = ">29"
    ys = [counter[x] for x in range(1, 30)]
    ys.append(len(counts) - sum(ys))
    ys = np.array(ys) / 1000
    plt.bar(xs + (i - 0.5) * 0.4, ys, width=0.4, label=name)
    # break
plt.xlim(0, 31)
plt.ylim(0, 10)
plt.text(5, 10 * 0.85, "isoform = %d" % s1, color="steelblue")
plt.text(5, 10 * 0.80, "mean = %.2f" % mean1, color="steelblue")
plt.text(5, 10 * 0.75, "median = %.2f" % median1, color="steelblue")
plt.text(5, 10 * 0.70, "isoform = %d" % s2, color="orange")
plt.text(5, 10 * 0.65, "mean = %.2f" % mean2, color="orange")
plt.text(5, 10 * 0.60, "median = %.2f" % median2, color="orange")
plt.xlabel("Isoform per gene")
plt.xticks(xs, xticks)
plt.ylabel("Gene number (x10$^3$)")
plt.legend()
plt.tight_layout()
if False:
    plt.show()
else:
    plt.savefig("figures/stats.isoform.count.png", dpi=300)
plt.close()

In [106]:
from pyBioInfo.IO.File import GtfFile, GtfTranscriptBuilder
from pyBioInfo.Utils import BlockTools

In [100]:
sites = []

In [101]:
with GtfFile("data/genome/annotation.gtf.gz") as f:
    records = [x for x in f]

In [103]:
transcripts = list(GtfTranscriptBuilder(records))

In [105]:
print(len(transcripts))

35307


In [122]:
sites1 = []
for transcript in transcripts:
    for start, end in  BlockTools.gaps(transcript.blocks):
        site = (transcript.chrom, start, end, transcript.strand)
        sites1.append(site)
sites1 = set(sites1)

In [125]:
len(sites1)

235083

In [150]:
sites2 = []
with open("../1_ngs/results/mapping/all.2nd.SJ.out.tab") as f:
    for line in f:
        chrom, start, end, strand, motif, annotate, uniq, multi, overhang = line.strip("\n").split("\t")
        start = int(start) - 1
        end = int(end)
        if strand == "1":
            strand = "+"
        elif strand == "2":
            strand = "-"
        else:
            strand = "."
        site = (chrom, start, end, strand)
        uniq = int(uniq)
        if uniq >= 1:
            site = (chrom, start, end, strand)
            sites2.append(site)
sites2 = set(sites2)
len(sites2)

1078503

In [151]:
sites = sites1 | sites2

In [152]:
len(sites)

1084630

In [136]:
with GtfFile("results/cupcake/chain/cupcake.gtf.gz") as f:
    records = [x for x in f]

In [139]:
transcripts = list(GtfTranscriptBuilder(records))
len(transcripts)

156798

In [153]:
for transcript in transcripts:
    introns = BlockTools.gaps(transcript.blocks)
    flags = []
    for intron in introns:
        site = (transcript.chrom, intron[0], intron[1], transcript.strand)
        flags.append(site in sites)
    transcript.flags = flags

In [161]:
array1 = []
array2 = []
for transcript in transcripts:
    flags = transcript.flags
    m = sum(flags)
    n = len(flags)
    if m == n or m / n > 0.8:
        array1.append(transcript)
    else:
        array2.append(transcript)
print(len(array1), len(array2))

147966 8832


In [160]:
with open("splice.junction.bed", "w+") as fw:
    for transcript in array1:
        fw.write(transcript.format("BED") + "\n")
assert os.system("bgzip -f splice.junction.bed") == 0
assert os.system("tabix -p bed -f splice.junction.bed.gz") == 0

In [158]:
with open("no.splice.junction.bed", "w+") as fw:
    for transcript in array2:
        fw.write(transcript.format("BED") + "\n")
assert os.system("bgzip -f no.splice.junction.bed") == 0
assert os.system("tabix -p bed -f no.splice.junction.bed.gz") == 0